In [2]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import os
import csv


## Open Dataset Files

In [18]:
# load .mat from dataset

filename =  r'Data/BCICIV_1calib_1000Hz_mat/BCICIV_calib_ds1a_1000Hz.mat'

# get data from .mat
def inputmat(fp):
    """load .mat file and return m as a dict"""
    mat = loadmat(fp, squeeze_me=True)
    td = {}  # create a dict

    # Numpy array of size channel_num * points.
    td['data'] = mat['cnt'].T  # data
    td['freq'] = mat['nfo']['fs'][True][0]  # Sampling frequency

    # channel names are necessary information for creating a rawArray.
    td['ch_names'] = mat['nfo']['clab'][True][0]

    # Position of channels
    
    td['electrode_x'] = mat['nfo']['xpos'][True][0]
    td['electrode_y'] = mat['nfo']['ypos'][True][0]

    #make trials by finding trials and its data
    td['cue'] = mat['mrk']['pos'][True][0] #time of cue
    td['labels'] = mat['mrk']['y'][True][0] #labels of the data
    return td


# filter data using bandpass 7 - 30 Hz
def filter_data(data_dict):
    """filter data from data_dict"""
    from scipy import signal
    data = data_dict['data']
    freq = data_dict['freq']
    # bandpass filter
    b, a = signal.butter(4, [7, 30], btype='bandpass', fs=freq)
    data = signal.filtfilt(b, a, data, axis=1)
    data_dict['data'] = data
    return data_dict


def data_to_csv(data_dict, averaging=False, filename='data.csv'):
    """convert data from data_dict to csv file for one channel if averaging enabled"""
    data = data_dict['data']
    labels = data_dict['labels']
    freq = data_dict['freq']
    cue = data_dict['cue']
    ch_names = data_dict['ch_names']
    if averaging:
        data = np.mean(data, axis=0)
        ch_names = ['Averaged']
        # create a new channel by averaging all the channels
        new_data = np.zeros((data.shape[1], 1))
        for i in range(data.shape[1]):
            new_data[i] = np.mean(data[:, i])
        data_dict['data'] = new_data
        data_dict['ch_names'] = ['Averaged']
    # write to csv
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(ch_names)
        for i in range(data.shape[1]):
            writer.writerow(data[:, i])
    return data_dict

data_dict = inputmat(filename)
data_to_csv(data_dict, averaging=False, filename='Data/data.csv')
data_to_csv(filter_data(data_dict), averaging=False, filename='Data/data_filtered.csv')

{'data': array([[ -5.82891176,  -0.31417617,   5.11461364, ...,  -8.51511564,
         -11.53862995, -14.09721601],
        [ -3.56396393,   6.18031822,  15.87041939, ..., -18.45458459,
         -21.12999136, -23.17731035],
        [ -8.10066709,  -0.74945111,   6.4480923 , ...,   6.82883187,
           3.40278894,   0.22328798],
        ...,
        [ 10.80629177,  19.59842754,  28.2243784 , ...,  -6.07530584,
          -8.54512885, -10.53684444],
        [ 11.45189126,  18.44528849,  25.26838276, ...,   1.49276505,
          -2.0614468 ,  -5.14262744],
        [ 13.19494725,  22.13718402,  30.84422849, ..., -14.78953707,
         -15.564213  , -15.97810012]]),
 'freq': 1000,
 'ch_names': array(['AF3', 'AF4', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'FC5',
        'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'CFC7', 'CFC5', 'CFC3',
        'CFC1', 'CFC2', 'CFC4', 'CFC6', 'CFC8', 'T7', 'C5', 'C3', 'C1',
        'Cz', 'C2', 'C4', 'C6', 'T8', 'CCP7', 'CCP5', 'CCP3', 'CCP1',
        'CCP2',